<a href="https://colab.research.google.com/github/chanukaDissanayaka/RideFareClassification/blob/master/Ridefare_submission_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#imports

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

from datetime import datetime, timedelta
from pandas import DataFrame

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier

import numpy as np

RANDOM_SEED = 4

In [0]:
dataframe = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [0]:
dataframe.isna().sum(axis = 0)
#dataframe = dataframe.fillna(method='ffill')
dataframe = dataframe.dropna()

In [0]:
dataframe.head()
dataframe.shape

(16968, 14)

In [0]:
df_majority = dataframe.loc[dataframe.label == "correct"].copy()
df_minority = dataframe.loc[dataframe.label == "incorrect"].copy()

In [0]:
df_majority.shape
df_minority.shape

(1526, 14)

In [0]:
#upsample
from sklearn.utils import resample

df_minority_upsampled = resample(df_minority,
                             replace=True,  # sample with replacement
                             n_samples=15000,  # to match majority class
                             random_state=123)  # reproducible results

In [0]:
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

In [0]:
df_upsampled.shape

(30442, 14)

In [0]:
print(df_upsampled.label.value_counts())

correct      15442
incorrect    15000
Name: label, dtype: int64


In [0]:
cleaned_train_set = preprocess_train_data(df_upsampled)
labels = get_labels(df_upsampled)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
print(cleaned_train_set.shape, labels.shape)

(30442, 20) (30442,)


In [0]:
cleaned_train_set.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_time_year,pickup_time_month,pickup_time_day,pickup_time_hour,pickup_time_minute,drop_time_year,drop_time_month,drop_time_day,drop_time_hour,drop_time_minute
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,2019,11,1,0,20,2019,11,1,0,34
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,2019,11,1,0,56,2019,11,1,1,9
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,2019,11,1,1,8,2019,11,1,1,26
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,2019,11,1,2,27,2019,11,1,2,37
5,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,2019,11,1,5,38,2019,11,1,6,35


In [0]:
cleaned_train_set = cleaned_train_set.drop('pickup_time_year',1)
cleaned_train_set = cleaned_train_set.drop('pickup_time_month',1)
cleaned_train_set = cleaned_train_set.drop('drop_time_year',1)
cleaned_train_set = cleaned_train_set.drop('drop_time_month',1)

In [0]:
scaler = get_minmax_scaler(cleaned_train_set)
train_set = scale_data(cleaned_train_set,scaler)

In [0]:
train_set[0]

array([8.20631136e-04, 1.83752247e-03, 1.23443183e-04, 0.00000000e+00,
       1.24785525e-03, 4.37643429e-01, 6.73666527e-02, 2.27882629e-02,
       9.68309899e-01, 1.00319194e-02, 0.00000000e+00, 0.00000000e+00,
       3.38983051e-01, 0.00000000e+00, 0.00000000e+00, 5.76271186e-01])

Split

In [0]:
X_train, X_eval, Y_train, Y_eval = train_test_split(
    train_set,
    labels,
    test_size=0.20,
    shuffle=True,
    stratify=labels,
    random_state=RANDOM_SEED
)

In [0]:
cleaned_test_data = preprocess_train_data(test)

cleaned_test_data = cleaned_test_data.drop('pickup_time_year',1)
cleaned_test_data = cleaned_test_data.drop('pickup_time_month',1)
cleaned_test_data = cleaned_test_data.drop('drop_time_year',1)
cleaned_test_data = cleaned_test_data.drop('drop_time_month',1)

test_set = scale_data(cleaned_test_data,scaler)

In [0]:

test_set.shape

(8576, 16)

In [0]:
print(X_eval.shape, X_train.shape)
print(labels.label.value_counts())

Model

In [0]:

rf = RandomForestClassifier(max_depth=40, random_state=0,)
rf.fit(X_train, Y_train) 
rf.score(X_eval, Y_eval)     

0.9904746263754312

In [0]:
tprf = RandomForestClassifier(bootstrap=False, criterion="gini", min_samples_leaf=8, min_samples_split=7, n_estimators=5)
tprf.fit(X_train, Y_train) 
tprf.score(X_eval, Y_eval)

0.9714238791262934

In [0]:
from sklearn.neural_network import MLPClassifier

mlpclf = MLPClassifier(random_state=1, max_iter=5000, verbose = True, alpha = 0.001,
                    hidden_layer_sizes = (20,8),solver = "lbfgs", early_stopping=True)
mlpclf.fit(X_train, Y_train)

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(20, 8), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=5000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [0]:
mlpclf.score(X_eval, Y_eval)

0.9228049499116088

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

gbclf = GradientBoostingClassifier(random_state=0, n_estimators = 420, max_depth =10, max_features='auto')
gbclf.fit(X_train, Y_train)
gbclf.score(X_eval, Y_eval)

0.9911315486943669

In [0]:

def define_model():
	model = Sequential()
	model.add(Dense(50, input_dim=16, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

In [0]:
model = define_model()
model.fit(X_train, Y_train, epochs=250, verbose=2)



In [0]:
#eval_predict = model.predict_classes(X_eval)
#print(eval_predict)
base = gbclf.predict(X_eval)
#print("keras", f1_score(Y_eval, eval_predict, average='micro'))
print("base", f1_score(Y_eval, base, average='micro'))

base 0.9911315486943669


In [0]:
from sklearn.metrics import confusion_matrix

results = gbclf.predict(X_eval)
confusion_matrix(Y_eval, results)

array([[3000,    0],
       [  54, 3035]])

In [0]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(Y_eval, results)

print('Average precision-recall score: {0:0.4f}'.format(
      average_precision))

Average precision-recall score: 0.9493


Test Data and predict

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

gbclf = GradientBoostingClassifier(random_state=0, n_estimators = 420, max_depth =10, max_features='auto')
gbclf.fit( train_set, labels)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=10,
                           max_features='auto', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=420,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
rf.fit(train_set, labels)

In [0]:
preds = gbclf.predict(test_set)


OUTPUT

In [0]:

output = pd.DataFrame(test,columns=['tripid'])
output['prediction'] = preds
output.head()


,tripid,prediction
0,213284604,1
1,213286352,1
2,213293973,1
3,213294622,1
4,213298687,1


In [0]:
output.to_csv('/content/Ridefare_submission_7.csv',index=False)

Preprocessing Functions

In [0]:
def preprocess_train_data(dataframe):
  feature_set = dataframe.iloc[:,1:13]

  features = feature_set

  features['pickup_time'] = pd.to_datetime(features['pickup_time'], format="%m/%d/%Y %H:%M")
  features['drop_time'] = pd.to_datetime(features['drop_time'], format="%m/%d/%Y %H:%M")

  td = features

  td['pickup_time_year'] = td['pickup_time'].dt.year
  td['pickup_time_month'] = td['pickup_time'].dt.month
  td['pickup_time_day'] = td['pickup_time'].dt.day
  td['pickup_time_hour'] = td['pickup_time'].dt.hour
  td['pickup_time_minute'] = td['pickup_time'].dt.minute

  td['drop_time_year'] = td['drop_time'].dt.year
  td['drop_time_month'] = td['drop_time'].dt.month
  td['drop_time_day'] = td['drop_time'].dt.day
  td['drop_time_hour'] = td['drop_time'].dt.hour
  td['drop_time_minute'] = td['drop_time'].dt.minute

  td = td.drop('pickup_time',1)
  td = td.drop('drop_time',1)

  return td

In [0]:
def get_labels(dataframe):
  label_set = dataframe.iloc[:, 13:14]

  rev_label_set = label_set.replace({'correct':'incorrect', 'incorrect':'correct'})

  encoder = LabelEncoder()
  encoder.fit(rev_label_set)
  labels = encoder.transform(rev_label_set)
  return labels

In [0]:
def get_scaler(train_data):
  scaler = StandardScaler()
  scaler.fit(train_data)
  return scaler

In [0]:
def get_minmax_scaler(train_data):
  scaler = MinMaxScaler()
  scaler.fit(train_data)
  return scaler

In [0]:

def get_maxabs_scaler(train_data):
  scaler = MaxAbsScaler()
  scaler.fit(train_data)
  return scaler

In [0]:
def scale_data(dataframe, scaler):
  scaled_dataframe = scaler.transform(dataframe)
  return scaled_dataframe

In [0]:
def crossFeatures(col1,col2,frame):
  newFrame = frame
  newFrame[col1+" X "+ col2] = frame[col1]*frame[col2]
  return newFrame